To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Llama-3 8b is trained on a crazy 15 trillion tokens! Llama-2 was 2 trillion.**

Use our [Llama-3 8b Instruct](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing) notebook for conversational style finetunes.

In [0]:
!nvidia-smi

Tue Jun 18 08:48:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000001:00:00.0 Off |                    0 |
| N/A   39C    P0              53W / 300W |      4MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [0]:
if 'dbutils' in locals():
    dbutils.library.restartPython()


In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import os
import sys
from pathlib import Path

def set_workding_dir(change_to_parent=False):
    workding_dir = Path.cwd()
    if change_to_parent:
        workding_dir = str(Path.cwd().parent)
        os.chdir(workding_dir)
    sys.path.append(workding_dir)
    print("workding dir:", workding_dir)

set_workding_dir(change_to_parent=True)

workding dir: /Workspace/Users/donghao.huang@mastercard.com/novel-translation


In [0]:
# need_to_setup_env = True
need_to_setup_env = False
need_to_setup_env = 'dbutils' in locals() and need_to_setup_env

if need_to_setup_env:
    %pip config set global.index-url https://artifacts.forge.mastercard.com/artifactory/api/pypi/python/simple
    %pip install --quiet torch torchvision --index-url https://download.pytorch.org/whl/cu121
    %pip install --quiet -r requirements.txt

    %restart_python
    from pathlib import Path
    print("workding dir:", Path.cwd())

In [0]:
# %sh FLASH_ATTENTION_FORCE_BUILD=TRUE pip install --quiet --upgrade flash-attn

In [0]:
from dotenv import find_dotenv, load_dotenv

found_dotenv = find_dotenv(".env")

if len(found_dotenv) == 0:
    found_dotenv = find_dotenv(".env.example")
print(f"loading env vars from: {found_dotenv}")
load_dotenv(found_dotenv, override=True)

loading env vars from: /Workspace/Users/donghao.huang@mastercard.com/novel-translation/.env


True

In [0]:
import os

model_name = os.getenv("MODEL_NAME")
token = os.getenv("HF_TOKEN") or None
load_in_4bit = os.getenv("LOAD_IN_4BIT") == "true"
local_model = os.getenv("LOCAL_MODEL")
hub_model = os.getenv("HUB_MODEL")
num_train_epochs = int(os.getenv("NUM_TRAIN_EPOCHS") or 0)
data_path = os.getenv("DATA_PATH")

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)

model_name, load_in_4bit, local_model, hub_model, max_seq_length, num_train_epochs, dtype, data_path

('unsloth/Qwen2-7B-Instruct',
 True,
 'models/Qwen2-7B-Instruct-',
 'Qwen2-7B-Instruct-MAC-',
 2048,
 10,
 None,
 'datasets/mac/mac.tsv')

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

In [0]:
%%time

from translation_engine import *

model, tokenizer = load_model(model_name, load_in_4bit)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f1178023600>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'
20

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


==((====))==  Unsloth: Fast Qwen2 patching release 2024.6
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.151 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 15.5 s, sys: 3.93 s, total: 19.4 s
Wall time: 15.9 s


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [0]:
%python
%%time

model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2024.6 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


CPU times: user 43.9 s, sys: 158 ms, total: 44 s
Wall time: 5.05 s


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [0]:
%python
import os
from translation_engine import *

datasets = load_translation_dataset(data_path, tokenizer)

loading train/test data files


/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:45: UserWarning: The cache_dir for this dataset is /root/.cache, which is not a persistent path.Therefore, if/when the cluster restarts, the downloaded dataset will be lost.The persistent storage options for this workspace/cluster config are: [DBFS, UC Volumes].Please update either `cache_dir` or the environment variable `HF_DATASETS_CACHE`to be under one of the following root directories: ['/dbfs/', '/Volumes/']
  warnings.warn(warning_message)
/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:14: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['chinese', 'english', 'text', 'prompt'],
        num_rows: 4528
    })
    test: Dataset({
        features: ['chinese', 'english', 'text', 'prompt'],
        num_rows: 1133
    })
})


In [0]:
datasets["train"][0], datasets["test"][0]

({'chinese': '全仗着狐仙搭救。',
  'english': 'Because I was protected by a fox fairy.',
  'text': '<|im_start|>system\nYou are an expert in translating Chinese to English.<|im_end|>\n<|im_start|>user\nPlease translate the following Chinese text into English and provide only the translated content, nothing else.\n全仗着狐仙搭救。<|im_end|>\n<|im_start|>assistant\nBecause I was protected by a fox fairy.<|im_end|>',
  'prompt': '<|im_start|>system\nYou are an expert in translating Chinese to English.<|im_end|>\n<|im_start|>user\nPlease translate the following Chinese text into English and provide only the translated content, nothing else.\n全仗着狐仙搭救。<|im_end|>\n<|im_start|>assistant\n'},
 {'chinese': '老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。',
  'english': 'Old Geng picked up his shotgun, squinted, and pulled the trigger. Two sparrows crashed to the ground like hailstones as shotgun pellets tore noisily through the branches.',
  'text': '<|im_start|>system\nYou are an expert in translating C

In [0]:
datasets["train"][100], datasets["test"][100]

({'chinese': '几分钟后，浏览器上出现了一个坐标窗口，一条红色曲线在窗口上出现，与另外三条进行着精确同步的波动。',
  'english': 'A few minutes later, a red waveform appeared in the browser window, moving in step with the other three.',
  'text': '<|im_start|>system\nYou are an expert in translating Chinese to English.<|im_end|>\n<|im_start|>user\nPlease translate the following Chinese text into English and provide only the translated content, nothing else.\n几分钟后，浏览器上出现了一个坐标窗口，一条红色曲线在窗口上出现，与另外三条进行着精确同步的波动。<|im_end|>\n<|im_start|>assistant\nA few minutes later, a red waveform appeared in the browser window, moving in step with the other three.<|im_end|>',
  'prompt': '<|im_start|>system\nYou are an expert in translating Chinese to English.<|im_end|>\n<|im_start|>user\nPlease translate the following Chinese text into English and provide only the translated content, nothing else.\n几分钟后，浏览器上出现了一个坐标窗口，一条红色曲线在窗口上出现，与另外三条进行着精确同步的波动。<|im_end|>\n<|im_start|>assistant\n'},
 {'chinese': '然后他们到一间为游客开的通宵酒吧中去坐了两个多小时，沙瑞山一杯接着一杯地灌啤酒，变得更加健谈，而汪淼却早已心神不定，脑

In [0]:
%%time

prompt1 = datasets["test"]["prompt"][100]
print(prompt1)
print("--" * 20)
test_model(model, tokenizer, prompt1)

<|im_start|>system
You are an expert in translating Chinese to English.<|im_end|>
<|im_start|>user
Please translate the following Chinese text into English and provide only the translated content, nothing else.
然后他们到一间为游客开的通宵酒吧中去坐了两个多小时，沙瑞山一杯接着一杯地灌啤酒，变得更加健谈，而汪淼却早已心神不定，脑子里不断地浮现出那条绿色直线。<|im_end|>
<|im_start|>assistant

----------------------------------------
<|im_start|>system
You are an expert in translating Chinese to English.<|im_end|>
<|im_start|>user
Please translate the following Chinese text into English and provide only the translated content, nothing else.
然后他们到一间为游客开的通宵酒吧中去坐了两个多小时，沙瑞山一杯接着一杯地灌啤酒，变得更加健谈，而汪淼却早已心神不定，脑子里不断地浮现出那条绿色直线。<|im_end|>
<|im_start|>assistant
Then they spent more than two hours at a 24-hour bar for tourists, where Sharisun drank beer one after another and became increasingly talkative. Meanwhile, Wang Miao was already restless, his mind constantly replaying that green line.<|im_end|>
CPU times: user 2.63 s, sys: 188 ms, total: 2.82 s
Wall time: 3.49 s


In [0]:
%%time

predictions = eval_model(model, tokenizer, datasets["test"])

 43%|████▎ 

*** WARNING: max output size exceeded, skipping output. ***

100%|██████████| 1133/1133 [33:04<00:00,  1.75s/it]

CPU times: user 33min 3s, sys: 3.89 s, total: 33min 6s
Wall time: 33min 4s


In [0]:
%python
calc_metrics(datasets["test"]["english"], predictions, debug=True)

{'accuracy': 0.00176522506619594,
 'correct_ids': [364, 659],
 'meteor': 0.37449710407725656,
 'bleu_scores': {'bleu': 0.11183638574583862,
  'precisions': [0.420306730097475,
   0.1509714443753025,
   0.07062929514625985,
   0.03551871577526067],
  'brevity_penalty': 0.995651386973693,
  'length_ratio': 0.9956608148393508,
  'translation_length': 30059,
  'reference_length': 30190},
 'rouge_scores': {'rouge1': 0.42145749325925663,
  'rouge2': 0.17369013267758654,
  'rougeL': 0.3640412935633255,
  'rougeLsum': 0.36417627213739723}}

In [0]:
%python
save_results(
    model_name,
    "results/mac-results.csv",
    datasets["test"],
    predictions,
    debug=True,
)

   index  ...                          unsloth/Qwen2-7B-Instruct
0      0  ...  Old Geng raised his gun, squinting one of his ...

[1 rows x 7 columns]


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [0]:
%python
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=datasets["train"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=num_train_epochs,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=100,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

[W CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())
torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


Map (num_proc=2):   0%|          | 0/4528 [00:00<?, ? examples/s]

In [0]:
# @title Show current memory stats
import torch

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100 80GB PCIe. Max memory = 79.151 GB.
15.078 GB of memory reserved.


In [0]:
%%time

trainer_stats = trainer.train()

---------------------------------------------------------------------------
The Python process exited with exit code 139 (SIGSEGV: Segmentation fault).



The last 10 KB of the process's stderr and stdout can be found below. See driver logs for full logs.
---------------------------------------------------------------------------
Last messages on stderr:
/.ipykernel/9250/command-3235276842469315-271335705", line 1 in <module>
  File "/databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526 in run_code
  File "/databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3466 in run_ast_nodes
  File "/databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3284 in run_cell_async
  File "/databricks/python/lib/python3.11/site-packages/IPython/core/async_helpers.py", line 129 in _pseudo_sync_runner
  File "/databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3079

In [0]:
%python
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [0]:
%python
%%time

prompt1 = datasets["test"]["prompt"][0]
print(prompt1)
print("--" * 20)
test_model(model, tokenizer, prompt1)

In [0]:
%python
print(datasets["test"]["english"][0])

In [0]:
%python
%%time

predictions = eval_model(model, tokenizer, datasets["test"])

In [0]:
%python
calc_metrics(datasets["test"]["english"], predictions, debug=True)

In [0]:
%python
from translation_engine import save_results

save_results(model_name + "(finetuned)", "results/mac-results.csv", datasets["test"], predictions, debug=True)

<a name="Save"></a>
### Saving, uploading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

In [0]:
%python
def save_model(model, tokenizer, save_method, publish=True):
    model.save_pretrained_merged(
        local_model + save_method,
        tokenizer,
        save_method=save_method,
    )

    if publish:
        model.push_to_hub_merged(
            hub_model + save_method,
            tokenizer,
            save_method=save_method,
            token=token,
        )

In [0]:
%python
save_model(model, tokenizer, "merged_4bit_forced")

In [0]:
def save_model_gguf(model, tokenizer, quantization_method, publish=True):
    model.save_pretrained_gguf(
        local_model + quantization_method,
        tokenizer,
        quantization_method=quantization_method,
    )

    if publish:
        model.push_to_hub_gguf(
            hub_model + "gguf-" + quantization_method,
            tokenizer,
            quantization_method=quantization_method,
            token=token,
        )

In [0]:
save_model_gguf(model, tokenizer, quantization_method="q4_k_m")

In [0]:
%python
%%time

# Empty VRAM
del model
del trainer

# clear memory
import torch
torch.cuda.empty_cache()

# garbage collect
import gc
gc.collect()
gc.collect()

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>